# FEM-BEM coupling of Poisson-Boltzmann

In [1]:
import numpy as np
from mpi4py import MPI
from dolfinx.fem import FunctionSpace
from dolfinx.mesh import create_unit_square, create_unit_cube
from dolfinx.geometry import (compute_colliding_cells,
                              compute_collisions_points)

def get_local_coordinates(vertices, point):
    """Get the local coordinates of the point in the cell."""
    origin = vertices[0]
    axes = [v - origin for v in vertices[1:]]
    tdim = 3
    if len(axes) == 2:
        axes.append(np.cross(axes[0], axes[1]))
        tdim = 2

    assert len(axes) == 3


    return np.linalg.solve(np.array(axes).T, point - origin)[:tdim]


def assemble_point_sources(space, points, weights):
    vector = np.zeros(space.dofmap.index_map.size_global)

    mesh = space.mesh
    for point, weight in zip(points, weights):
        # Get cell
        tree = dolfinx.geometry.bb_tree(mesh, mesh.geometry.dim)
        cell_candidates = compute_collisions_points(tree, point)
        # This gets the first cell. Would it be better to put a fraction of the delta function into each cell instead?
        cell = compute_colliding_cells(mesh, cell_candidates, point)[0]

        # Get local coordinates
        # Note: this currently only works for affine triangles and tetrahedra
        v = [mesh.geometry.x[i] for i in mesh.geometry.dofmap[cell]]
        local_coordinates = get_local_coordinates(v, point)

        # Note: this currently only works for scalar-valued elements that use an identity push forward map
        values = space.element.basix_element.tabulate(0, [local_coordinates])[0, 0, :, 0]
        dofs = space.dofmap.cell_dofs(cell)

        for d, v in zip(dofs, values):
            vector[d] += v * weight

    return vector


def test_point_sources_triangle():
    points = np.array([(1/4, 1/6, 0), (1/10, 6/10, 0)])

    mesh = create_unit_square(MPI.COMM_WORLD, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [1, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 4/5)
    assert np.isclose(a[-2], 1/2)
    assert np.isclose(a[-3], 1/3)
    assert np.isclose(a[-4], 1/5)
    assert np.isclose(a[-5], 1/6)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)

    vector = assemble_point_sources(space, points, [2, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 1)
    assert np.isclose(a[-2], 4/5)
    assert np.isclose(a[-3], 2/3)
    assert np.isclose(a[-4], 1/3)
    assert np.isclose(a[-5], 1/5)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)


def test_point_sources_tetrahedron():
    points = np.array([(1/4, 1/6, 0), (1/2, 1/2, 15/16)])

    mesh = create_unit_cube(MPI.COMM_WORLD, 2, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [1, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 7/8)
    assert np.isclose(a[-2], 1/2)
    assert np.isclose(a[-3], 1/3)
    assert np.isclose(a[-4], 1/6)
    assert np.isclose(a[-5], 1/8)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)


    mesh = create_unit_cube(MPI.COMM_WORLD, 2, 2, 2)
    space = FunctionSpace(mesh, ("Lagrange", 1))

    vector = assemble_point_sources(space, points, [2, 1])

    a = [i for i in vector]
    a.sort()
    assert np.isclose(a[-1], 1)
    assert np.isclose(a[-2], 7/8)
    assert np.isclose(a[-3], 2/3)
    assert np.isclose(a[-4], 1/3)
    assert np.isclose(a[-5], 1/8)
    assert np.isclose(a[-6], 0)
    assert np.isclose(a[0], 0)

## Solving problems for each of the mesh

In [2]:
from mpi4py import MPI 
from dolfinx.mesh import create_unit_cube

def poisson_boltzman_jn_const(mesh):
    
    nor = ufl.FacetNormal(mesh) 
    
    # Spaces for FEM and BEM #
    fenics_space = FunctionSpace(mesh, ("Lagrange", 1))
    trace_space, trace_matrix = \
    fenicsx.fenics_to_bempp_trace_data(fenics_space)
    bempp_space = bempp.api.function_space(trace_space.grid, "P", 1)

    # Mesh size information #
    fem_size = fenics_space.dofmap.index_map.size_global
    print("FEM dofs: {0}".format(fem_size))
    print("BEM dofs: {0}".format(bempp_space.global_dof_count))
    hmax = trace_space.grid.maximum_element_diameter
    
    # Starting set-up time
    start = time.time()     
    
    # FEM discrete variables
    u = ufl.TrialFunction(fenics_space)
    v = ufl.TestFunction(fenics_space)
        
    # BEM operators
    id_op = bempp.api.operators.boundary.sparse.identity(trace_space, bempp_space, bempp_space)
    mass = bempp.api.operators.boundary.sparse.identity(bempp_space, bempp_space, trace_space)
    
    if fmm:
        dlp_0 = bempp.api.operators.boundary.laplace.double_layer(trace_space, bempp_space, bempp_space, assembler="fmm")
        slp_0 = bempp.api.operators.boundary.laplace.single_layer(bempp_space, bempp_space, bempp_space, assembler="fmm")

        dlp = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space, bempp_space, bempp_space, w, assembler="fmm")
        slp = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space, bempp_space, bempp_space, w, assembler="fmm")
    else:
        dlp_0 = bempp.api.operators.boundary.laplace.double_layer(trace_space, bempp_space, bempp_space)
        slp_0 = bempp.api.operators.boundary.laplace.single_layer(bempp_space, bempp_space, bempp_space)

        dlp = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space, bempp_space, bempp_space, w)
        slp = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space, bempp_space, bempp_space, w)

    ######################### Matrix and RHS ##################################
    
    # RHS for FEM
    rhs_fem = assemble_point_sources(fenics_space, PC, Q/em)
        
    # RHS for BEM
    rhs_bem = np.zeros(bempp_space.global_dof_count)
    # Global RHS
    rhs = np.concatenate([rhs_fem, rhs_bem])
    
    blocks = [[None,None],[None,None]]
    
    # FEM matrices
    trace_op = LinearOperator(trace_matrix.shape, lambda x:trace_matrix*x)

    A = fenicsx.FenicsOperator((ufl.inner(ufl.nabla_grad(u),
                                     ufl.nabla_grad(v)) ) * ufl.dx)  
        
    # Blocked matrix
    blocks[0][0] = A.weak_form()
    blocks[0][1] = -trace_matrix.T * mass.weak_form().to_sparse()
    blocks[1][0] = (.5 * id_op - dlp).weak_form() * trace_op
    blocks[1][1] = slp.weak_form()*(em/es)

    blocked = BlockedDiscreteOperator(np.array(blocks))
    
    ######################### Preconditioner and solvers ##################################
    from scipy.sparse import diags
    
    # Diagonal Scaling #
    P1 = diags(1./blocked[0,0].to_sparse().diagonal())

    # Mass matrix preconditioner #
    P2 = InverseSparseDiscreteBoundaryOperator(
        bempp.api.operators.boundary.sparse.identity(
            bempp_space, bempp_space, bempp_space).weak_form())

    # Create a block diagonal preconditioner object using the Scipy LinearOperator class
    def apply_prec(x):
        """Apply the block diagonal preconditioner"""
        m1 = P1.shape[0]
        m2 = P2.shape[0]
        n1 = P1.shape[1]
        n2 = P2.shape[1]

        res1 = P1.dot(x[:n1]) 
        res2 = P2.dot(x[n1:])
        return np.concatenate([res1, res2])

    p_shape = (P1.shape[0] + P2.shape[0], P1.shape[1] + P2.shape[1])
    P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('float64'))
        
    end = time.time()
    set_time = (end - start) 
    

    from scipy.sparse.linalg import gmres
    start = time.time()     
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations, tol=tol, restrt=500)  
    end = time.time()
    solv_time = (end - start) 
    
    soln_fem = soln[:fem_size]
    soln_bem = soln[fem_size:]
    
    u = dolfinx.fem.Function(fenics_space)
    u.vector.setValues(list(range(0,fem_size)), np.ascontiguousarray(soln_fem))
    dirichlet_data = trace_matrix * soln_fem
    dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=dirichlet_data)
    neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=soln_bem)
    
    ######################### Solvation energy ##################################
    
    slpF = bempp.api.operators.potential.laplace.single_layer(bempp_space, np.transpose(PC)) 
    dlpF = bempp.api.operators.potential.laplace.double_layer(trace_space, np.transpose(PC))

    uF = slpF * neumann_fun - dlpF * dirichlet_fun

    q_uF = 0
    for i in range(len(PC)):
        Sum = (uF[0][i].real)*Q[i]
        q_uF = q_uF + Sum

    E_Solv = 0.5*4.*np.pi*332.064*(q_uF) 
    
    print('Energia de Solvatacion : {:7.3f} [kCal/mol]'.format(E_Solv))
    
    return E_Solv, hmax, iter_num, set_time, solv_time

In [3]:
import dolfinx
import dolfinx.io
import dolfinx.geometry
from petsc4py import PETSc
import ufl
from scipy.sparse import csr_matrix

import bempp.api
from bempp.api.external import fenicsx
import numpy as np
import pylab as plt
import time

from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import LinearOperator
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator

from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
from bempp.api.operators.potential import laplace as lp

bempp.api.set_logging_level(level='info')
 
# Solver data #
tol = 1e-05         # Tolerance
fmm = 1             # Use FMM

# Problem data #
from readpqr_all import readpqr_all
w = 0.10265                                            # kappa
es = 80.                                               # External permitivity (solvant)
em = 1.                                                # Internal permitivity (molecule)

ec = 1.602176e-19 #[C]
kb = 1.380648e-23 #[J/K]
T  = 298 #[K]
S1 = 1 #(ec/(kb*T))  #38.681 [C/J]

# Result collecting lists #
solv_error = []
mesh_size = []
iter_num = []
time_solve = []
time_set = []

# Iteratotion for mesh #
grid = ['1a1t', '1biv', '1exy', '1g70', '1hji', '1i9f', '1mnb', '1nyb', '1qfq', '1ull', '1zbn', '2a9x', '484d']

it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1

for m in grid:
    print(m+'_protein')
    ######################### Mesh and spaces ##################################
    [PC, Q, R] = readpqr_all('./data-set3/'+m+'_protein.pqr')         # Charges, Points of charges and Radiuses
    mesh_file = "./data-set3/"+m+"_protein-converted.xdmf"
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, mesh_file, "r") as xdmf:
        mesh = xdmf.read_mesh(name="Grid")
        it_count = 0
        E_Solv, hmax, iter_num, set_time, solv_time = poisson_boltzman_jn_const(mesh)
        time_set.append(set_time)
        print("Setup time: {:10.4f}".format(set_time)) 
        time_solve.append(solv_time)   
        print("Solve time: {:10.4f}".format(solv_time)) 
        bempp.api.log(
            "gmres finished in %i iterations and took %.2E sec." % (it_count, solv_time))
        # print("Number of iterations: {0}".format(it_count))
        iter_num.append(it_count)
        mesh_size.append(hmax)   
        
        #### Just saving the solution
        solv_error.append(E_Solv)  

102ddna.pqr


INFO:bempp:Created grid with id a59319e7-1f2b-4e72-97a4-a58e5a9b40ff. Elements: 137020. Edges: 205530. Vertices: 68512


FEM dofs: 230022
BEM dofs: 68512


INFO:bempp:OpenCL CPU Device set to: pthread-AMD Ryzen Threadripper PRO 3975WX 32-Cores
66 warnings generated.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])

66 warnings generated.
  prg.build(options_bytes, [devices[i] for i in to_be_built_indices])

66 warnings generated.
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)

66 warnings generated.
66 warnings generated.
66 warnings generated.
66 warnings generated.
INFO:bempp:gmres finished in 215 iterations and took 1.74E+03 sec.


Energia de Solvatacion : -6236.301 [kCal/mol]
Setup time:   537.9519
Solve time:  1739.7157
109ddna.pqr


...

Energia de Solvatacion : -6211.669 [kCal/mol]
Setup time:   538.1762
Solve time:  1867.6053
360ddna.pqr


INFO:bempp:Created grid with id 7be7caf4-032d-412f-82d5-821315c9d9d3. Elements: 137892. Edges: 206838. Vertices: 68948


FEM dofs: 231551
BEM dofs: 68948


INFO:bempp:gmres finished in 211 iterations and took 1.73E+03 sec.


Energia de Solvatacion : -6211.669 [kCal/mol]
Setup time:   541.1241
Solve time:  1726.1883
